In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["LANGCHAIN_PROJECT"] = "lx"

print(f'LANGCHAIN_TRACING_V2: {os.getenv("LANGCHAIN_TRACING_V2", "未设置")}')
print(f'LANGCHAIN_ENDPOINT: {os.getenv("LANGCHAIN_ENDPOINT", "未设置")}')
print(f'LANGCHAIN_API_KEY: {os.getenv("LANGCHAIN_API_KEY", "未设置")}')
print(f'LANGCHAIN_PROJECT: {os.getenv("LANGCHAIN_PROJECT", "未设置")}')

LANGCHAIN_TRACING_V2: True
LANGCHAIN_ENDPOINT: https://api.smith.langchain.com
LANGCHAIN_API_KEY: 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
LANGCHAIN_PROJECT: lx


In [2]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_fireworks import ChatFireworks

# Initialize a Fireworks chat model
chat = ChatFireworks(model="accounts/fireworks/models/mixtral-8x7b-instruct")

# ChatFireworks Wrapper
system_message = SystemMessage(content="You are to chat with the user.")
human_message = HumanMessage(content="Who are you?")

chat.invoke([system_message, human_message])

AIMessage(content="Hello! I'm an AI language model, a helpful assistant here to chat with you and provide information, answer questions, and engage in conversation on a wide range of topics. How can I assist you today?", response_metadata={'token_usage': {'prompt_tokens': 21, 'total_tokens': 65, 'completion_tokens': 44}, 'model_name': 'accounts/fireworks/models/mixtral-8x7b-instruct', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='run-5ed128ee-c9b1-4a4a-a07f-16f23db33d44-0')

In [3]:
# Setting additional parameters: temperature, max_tokens, top_p
chat = ChatFireworks(
    model="accounts/fireworks/models/mixtral-8x7b-instruct",
    temperature=1,
    max_tokens=20,
)
system_message = SystemMessage(content="You are to chat with the user.")
human_message = HumanMessage(content="How's the weather today?")
chat.invoke([system_message, human_message])

AIMessage(content="I'm an AI and don't have real-time access to weather data, so I", response_metadata={'token_usage': {'prompt_tokens': 24, 'total_tokens': 44, 'completion_tokens': 20}, 'model_name': 'accounts/fireworks/models/mixtral-8x7b-instruct', 'system_fingerprint': '', 'finish_reason': 'length', 'logprobs': None}, id='run-1127cee0-65dc-4a9c-80a5-104a831cf071-0')

In [5]:
# FireFunction-v2
from pprint import pprint
from langchain_core.pydantic_v1 import BaseModel


class ExtractFields(BaseModel):
    name: str
    age: int

chat = ChatFireworks(
    model="accounts/fireworks/models/firefunction-v2",
).bind_tools([ExtractFields])

result = chat.invoke("我27岁，名字叫'我是你爹'")

pprint(result.additional_kwargs["tool_calls"][0])

{'function': {'arguments': '{"name": "我是你爹", "age": 27}',
              'name': 'ExtractFields'},
 'id': 'call_LJTd16hVS9ktVz4k1d0wopFV',
 'index': 0,
 'type': 'function'}


In [1]:
# 跟踪聊天示例
import os
from langchain_core.output_parsers import StrOutputParser
from langsmith.run_helpers import traceable
from langchain_core.prompts import PromptTemplate
from langchain_fireworks import ChatFireworks
from dotenv import load_dotenv

load_dotenv()
 
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
 
chat = ChatFireworks(
    model="accounts/fireworks/models/mixtral-8x7b-instruct",
    temperature=1,
)

@traceable  # Auto-trace this function
def pipeline(user_input: str):
    chain = PromptTemplate.from_template("你是一个导游专家，回答下边的问题：{user_input}") | chat | StrOutputParser() 
    return chain.invoke(user_input)
 

print(pipeline("我家在北京，请推荐一下特色美食？3"))


当然，以下是几个北京特色的美食建议：

1. 烤鸭 (Peking Duck)：这是新华国际（Quanjude）和四季春（Bianyifang）等著名餐厅的必吃菜肴。

2. 鼓浪屿炸酱面 (Gu LAO Ji Zha Jiang Mian)：国内外闻名的老北京面食，最令人难忘的口感之一。

3. 炸酱denly (Zha Jiang Mian)：这是另一种口感风格不同的老北京炸酱面，经典且香辣。

4. 面条捞鸡窝 (Jiaozi Wan Ji)：是一种在汤里面捞鸡窝汤饼的北京方式，不一定每家餐厅都有。

5. 糍子馒头 (Mantou)：最简单的北京烤面包，没有油的汤饼，常吃在清淡食物。

6. 醋溜子 (Cu Liu Zi)：用米粉制成，老北京人最爱吃的主食之一。

7. 豆腐脑 (Doufu Naod)：豆腐与豆腐皮烧炸后，混合鲜嫩的牛肉与猪心，制成淋入甜辣香鲜的酱汁的美味小吃。

8. 炸酱牛肉手撕包子 (Zha Jiang Niu Rou Shou Si Baozi)：牛肉鲜甜，与酱汁的搭配味道特别香味。

9. 凉拌糯米藜米 (Liang Ban Nuo Mi Mi)：清爽可口的食物，适配北京的热夏天。

10. 姥姥黄 (Laolao Huang)：热销的老北京甜品，是用糯米糍为基础，加入松子、花生、糖浆等混合制成。

您一定能在北京享受到美味的特色美食！
